## Deliverable 3. Create a Travel Itinerary Map.

In [24]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [25]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,2,Garliava,LT,71.58,few clouds,54.8167,23.8667,"Armenia, cafe-motel, TIR parking"
1,6,Kavaratti,IN,83.98,light rain,10.5669,72.6420,Hotels in Lakshadweep Islands
2,12,Puerto Ayora,EC,75.54,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
3,13,Torbay,CA,72.37,few clouds,47.6666,-52.7314,Gallows Cove
4,17,Tricase,IT,76.35,clear sky,39.9304,18.3553,Hotel Adriatico


In [26]:
# 1a. Drop the rows for any nulls.
clean_hotel_df = vacation_df.dropna(how='any')
clean_hotel_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,2,Garliava,LT,71.58,few clouds,54.8167,23.8667,"Armenia, cafe-motel, TIR parking"
1,6,Kavaratti,IN,83.98,light rain,10.5669,72.6420,Hotels in Lakshadweep Islands
2,12,Puerto Ayora,EC,75.54,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
3,13,Torbay,CA,72.37,few clouds,47.6666,-52.7314,Gallows Cove
4,17,Tricase,IT,76.35,clear sky,39.9304,18.3553,Hotel Adriatico
...,...,...,...,...,...,...,...,...
270,686,Calama,CL,70.29,clear sky,-22.4667,-68.9333,Diego De Almagro Calama
271,687,Weligama,LK,79.39,broken clouds,5.9667,80.4167,Weligama Bay Resort
272,689,Jasper,US,77.85,clear sky,37.2001,-94.3502,Guest House Motel
273,691,Dawei,MM,81.36,broken clouds,14.0833,98.2000,Hotel Dawei


In [27]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [8]:
# 4a. Add a marker layer for each city to the map.
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [31]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df['City'] == 'Visp']
vacation_end = vacation_df.loc[vacation_df['City'] == 'Visp']
vacation_stop1 = vacation_df.loc[vacation_df['City'] == 'Royan']
vacation_stop2 = vacation_df.loc[vacation_df['City'] == 'Den Helder'] 
vacation_stop3 = vacation_df.loc[vacation_df['City'] == 'Wolow'] 

In [32]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start['Lat'].to_numpy()[0],vacation_start['Lng'].to_numpy()[0]
end = vacation_end['Lat'].to_numpy()[0],vacation_end['Lng'].to_numpy()[0]
stop1 = vacation_stop1['Lat'].to_numpy()[0],vacation_stop1['Lng'].to_numpy()[0]
stop2 = vacation_stop2['Lat'].to_numpy()[0],vacation_stop2['Lng'].to_numpy()[0]
stop3 = vacation_stop3['Lat'].to_numpy()[0],vacation_stop3['Lng'].to_numpy()[0]

print(f'Start: {start}, End: {end}')
print(f'First stop: {stop1}, Second stop: {stop2}, Last stop: {stop3}')

Start: (46.2937, 7.8815), End: (46.2937, 7.8815)
First stop: (45.6285, -1.0281), Second stop: (52.9599, 4.7593), Last stop: (51.3366, 16.6443)


In [36]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".

fig = gmaps.figure()
vacation_trip = gmaps.directions_layer(
        start, end, waypoints=[stop1,stop2,stop3],
        travel_mode='DRIVING')

fig.add_layer(vacation_trip)
fig


Figure(layout=FigureLayout(height='420px'))

In [37]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start,vacation_stop1,vacation_stop2,vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,172,Visp,CH,75.02,overcast clouds,46.2937,7.8815,Baxter Hotel
1,591,Royan,FR,75.24,light rain,45.6285,-1.0281,Hôtel Cordouan Thalassothérapie et Spa - Royan
2,275,Den Helder,NL,74.98,haze,52.9599,4.7593,Hotel Den Helder
3,502,Wolow,PL,73.51,clear sky,51.3366,16.6443,LUNA noclegi


In [38]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [39]:
# 11a. Add a marker layer for each city to the map.

max_temp = itinerary_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))